In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22658600
paper_name = 'witkin_cohen_fix_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [9]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Table 1', skiprows=1)

In [10]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 35 x 4


In [11]:
original_data.head()

,Unnamed: 0,Gene,Functiona,Nuclear extensions mainly\nin large-budded cellsb
0,DNA recombination and repair,MMS1*,Resolves replication intermediates,yes
1,NaN,POL32*,DNA replication and error-prone DNA synthesis,yes
2,NaN,RAD27*,Okazaki fragment processing and long-patch bas...,yes
3,NaN,RAD34*,Nucleotide excision repair,yes
4,NaN,RAD5*,Post-replication repair,yes


In [12]:
original_data['genes'] = original_data['Gene'].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [14]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Gene, Functiona, Nuclear extensions mainly
in large-budded cellsb, genes, orf]
Index: []


In [16]:
original_data['data'] = 1

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(35, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [11848]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,11848
data_type,value
orf,
YBR026C,1
YBR221C,1
YCL029C,1
YDL013W,1
YDR076W,1


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11848
,data_type,value
gene_id,orf,
222,YBR026C,1
415,YBR221C,1
516,YCL029C,1
1892,YDL013W,1
2203,YDR076W,1


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      11848           
data_type       value     valuez
gene_id orf                     
222     YBR026C     1  11.405512
415     YBR221C     1  11.405512
516     YCL029C     1  11.405512
1892    YDL013W     1  11.405512
2203    YDR076W     1  11.405512

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 22658600...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  6.06it/s]


Updating the data_modified_on field...
